In [1]:
import os
import json
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    ExtraTreesClassifier,
    BaggingClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")

try:
    from xgboost import XGBClassifier
    xgboost_available = True
except ImportError:
    xgboost_available = False

In [2]:
def get_model_class(model_name):
    model_classes = {
        'LogisticRegression': LogisticRegression,
        'RandomForest': RandomForestClassifier,
        'GradientBoosting': GradientBoostingClassifier,
        'AdaBoost': AdaBoostClassifier,
    }
    if xgboost_available:
        model_classes['XGBoost'] = XGBClassifier
    return model_classes.get(model_name)

In [3]:
def get_models():
    models = {
        'LogisticRegression': {'params': {'max_iter': 1000, 'solver': 'lbfgs'}},
        'RandomForest': {'params': {'n_estimators': 100}},
        'GradientBoosting': {'params': {'n_estimators': 100}},
        'AdaBoost': {'params': {'n_estimators': 100}},
        
    }
    if xgboost_available:
        models['XGBoost'] = {'params': {'use_label_encoder': False, 'eval_metric': 'logloss'}}
    return models

In [4]:

def load_folder_list(json_file):
    with open(json_file, 'r') as f:
        folder_list = json.load(f)
    return folder_list

In [5]:
def train_and_evaluate(args):
    model_name, model_params, folder_path = args
    try:
        print(f"Starting {model_name} on {folder_path}")
        # Paths to scaled datasets
        train_file = os.path.join(folder_path, 'train_scaled.csv')
        test_file = os.path.join(folder_path, 'test_scaled.csv')
        
        # Check if files exist
        if not os.path.exists(train_file) or not os.path.exists(test_file):
            print(f"Scaled data not found in {folder_path}. Skipping...")
            return None
        
        # Load datasets
        train_data = pd.read_csv(train_file)
        test_data = pd.read_csv(test_file)
        
        # Separate features and target
        X_train = train_data.drop(columns=['target'])
        y_train = train_data['target']
        X_test = test_data.drop(columns=['target'])
        y_test = test_data['target']
        
        # Get model class
        model_class = get_model_class(model_name)
        if model_class is None:
            print(f"Model class for {model_name} not found.")
            return None
        
        # Instantiate the model
        model = model_class(**model_params['params'])
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        if hasattr(model, "predict_proba"):
            y_proba = model.predict_proba(X_test)[:,1]
        else:
            # For models without predict_proba
            y_proba = model.decision_function(X_test)
            # Normalize scores to [0,1]
            y_proba = (y_proba - y_proba.min()) / (y_proba.max() - y_proba.min())
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        try:
            auc = roc_auc_score(y_test, y_proba)
        except ValueError:
            auc = np.nan  # Handle cases where AUC cannot be computed
        
        print(f"Completed {model_name} on {folder_path}")
        # Return results as a dictionary
        result = {
            'model_name': model_name,
            'folder_path': folder_path,
            'accuracy': accuracy,
            'f1_score': f1,
            'auc_roc': auc
        }
        return result
    except Exception as e:
        print(f"Error in {model_name} on {folder_path}: {e}")
        return None

In [6]:
def run_models_on_datasets(folder_list, models):
    import time

    # Prepare arguments for each task
    tasks = []
    for model_name, model_params in models.items():
        for folder_path in folder_list:
            tasks.append((model_name, model_params, folder_path))

    total_tasks = len(tasks)
    completed_tasks = 0
    results = []

    # Initialize a ThreadPoolExecutor
    max_workers = max((os.cpu_count() *2) - 4 ,os.cpu_count())
    print("max_workers",max_workers)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks to the executor
        future_to_task = {executor.submit(train_and_evaluate, args): args for args in tasks}
        
        # Collect results as they complete
        for future in as_completed(future_to_task):
            result = future.result()
            if result is not None:
                results.append(result)
            completed_tasks += 1
            print(f"Completed {completed_tasks}/{total_tasks} tasks")
    return results

In [7]:
def get_top_performers(results, top_n=5):
    # Organize results by model
    model_results = defaultdict(list)
    for res in results:
        model_results[res['model_name']].append(res)
    
    # Find top N datasets for each model based on F1 score
    top_performers = {}
    for model_name, res_list in model_results.items():
        # Sort results by F1 score in descending order
        sorted_res = sorted(res_list, key=lambda x: x['f1_score'], reverse=True)
        top_performers[model_name] = sorted_res[:top_n]
    
    return top_performers

In [8]:

def main():
    # Load the list of folders
    folder_list = load_folder_list('final_folders_list.json')  # Replace with your JSON file
    
    # Get the models
    models = get_models()
    
    # Run models on datasets
    print("Starting model training and evaluation...")
    results = run_models_on_datasets(folder_list, models)
    print("Training and evaluation completed.")
    
    # Get top performers
    top_performers = get_top_performers(results, top_n=5)
    
    # Print the top performers
    for model_name, top_list in top_performers.items():
        print(f"\nTop 5 datasets for model {model_name} based on F1 score:")
        for idx, res in enumerate(top_list, 1):
            print(f"{idx}. Folder: {res['folder_path']}, F1 Score: {res['f1_score']:.4f}, Accuracy: {res['accuracy']:.4f}, AUC-ROC: {res['auc_roc']:.4f}")
    
    # Return the top performers if needed
    return top_performers


In [9]:




if __name__ == "__main__":
    top_performers = main()


Starting model training and evaluation...
max_workers 44
Starting LogisticRegression on handeled_null_values\10_threshold\using_mean_no_imbalance_no_reduction
Starting LogisticRegression on handeled_null_values\10_threshold\using_mean_no_imbalance_pca
Starting LogisticRegression on handeled_null_values\10_threshold\using_mean_no_imbalance_correlation
Starting LogisticRegression on handeled_null_values\10_threshold\using_mean_undersampling_no_reduction
Starting LogisticRegression on handeled_null_values\10_threshold\using_mean_undersampling_pca
Starting LogisticRegression on handeled_null_values\10_threshold\using_mean_undersampling_correlation
Starting LogisticRegression on handeled_null_values\10_threshold\using_mean_smote_no_reduction
Starting LogisticRegression on handeled_null_values\10_threshold\using_mean_smote_pca
Starting LogisticRegression on handeled_null_values\10_threshold\using_mean_smote_correlation
Starting LogisticRegression on handeled_null_values\10_threshold\using_me

In [10]:
top_performers


{'LogisticRegression': [{'model_name': 'LogisticRegression',
   'folder_path': 'handeled_null_values\\10_threshold\\using_mean_no_imbalance_no_reduction',
   'accuracy': 0.9694841675178754,
   'f1_score': 0.8525380947929326,
   'auc_roc': 0.9890117983973786},
  {'model_name': 'LogisticRegression',
   'folder_path': 'handeled_null_values\\10_threshold\\using_median_no_imbalance_no_reduction',
   'accuracy': 0.9695038812537766,
   'f1_score': 0.85180793193668,
   'auc_roc': 0.9889919101027862},
  {'model_name': 'LogisticRegression',
   'folder_path': 'handeled_null_values\\10_threshold\\using_mean_no_imbalance_correlation',
   'accuracy': 0.9692288049029623,
   'f1_score': 0.8515871386716928,
   'auc_roc': 0.9889821718555989},
  {'model_name': 'LogisticRegression',
   'folder_path': 'handeled_null_values\\10_threshold\\using_median_no_imbalance_correlation',
   'accuracy': 0.9692675973412249,
   'f1_score': 0.8509038974705905,
   'auc_roc': 0.9889638437167537},
  {'model_name': 'Logistic

### **saving top performer**

In [11]:
with open("top_performers.json","w") as file:
    json.dump(top_performers, file)